<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Guided Problem Set 6: Matched Filtering</h1>


<a name='section_6_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P6.0 Overview</h2>


<table style="width:100%">
    <colgroup>
       <col span="1" style="width: 40%;">
       <col span="1" style="width: 15%;">
       <col span="1" style="width: 45%;">
    </colgroup>
    <tr>
        <th style="text-align: left; font-size: 13pt;">Section</th>
        <th style="text-align: left; font-size: 13pt;">Problems</th>
        <th style="text-align: left; font-size: 13pt;">Summary</th>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_1">P6.1 What is Matched Filtering?</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_6_1">P6.1 Problems</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_2">P6.2 Time Domain</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_6_2">P6.2 Problems</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_3">P6.3 Frequency Domain</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_6_3">P6.3 Problems</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">
            <ul>
                <li>text</li>
                <li>text</li>
            </ul>
        </td>
    </tr>
</table>



<h3>Learning Objectives</h3>

In this recitation we will explore the following objectives:

- What is matched filtering?
- Matched filtering in the time domain
- Matched filtering in the frequency domain


<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook. 
Optionally, set the plot resolution and default figure size.


In [7]:
#>>>RUN

import numpy as np

#set plot resolution
#%config InlineBackend.figure_format = 'retina'

#set default figure size
#plt.rcParams['figure.figsize'] = (9,6)


<a name='section_6_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P6.1 What is Matched Filtering?</h2>    

| [Top](#section_6_0) | [Previous Section](#section_6_0) | [Problems](#problems_6_1) | [Next Section](#section_6_2) |


<h3>Overview</h3>

The purpose of matched filtering is to scan big data sets looking for some kind of signal. LIGO does this to look for gravitational waves in their strain data. Matched filtering is also done in many other fields.

The purpose is usually to create some kind of plot of signal to noise ratio (SNR) over your data. For LIGO, this is a 2D plot with time on the x axis and SNR on the y axis. If you're looking for point sources in astrophysical telescope data, for example, this is an image plot with right ascention and declination as the axes and SNR shown in the image.

Signals look like large spikes in the SNR.

To make this exercise useful to you in the LIGO project, we'll make a model signal that looks kind of like a black hole waveform. Run the below code to load the waveform and plot an example. (This is nearly the same function as was used in recitation 3)

In [10]:
#>>>RUN

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0x98a09fe)

def complicated_model_fn(x, time, lambda_plus, lambda_minus, max_amp, omega_0, omega_max, omega_sigma):
    omega = (omega_max - omega_0) * (np.exp(-np.minimum(x - time, 0)**2 / omega_sigma)) + omega_0
    lambdas = np.array([lambda_plus if xvalue > time else lambda_minus for xvalue in x])
    amplitude = max_amp * np.exp(-abs(x - time) / lambdas)
    return amplitude * np.cos(omega * (x-time))

LAMBDA_PLUS_TRUE = 1.0
LAMBDA_MINUS_TRUE = 4
MAX_AMP_TRUE = 1.2
OMEGA_0_TRUE = 3.0
OMEGA_MAX_TRUE = 6.0
OMEGA_SIGMA_TRUE = 4.0
TIME_TRUE = 50.0

xi = np.linspace(TIME_TRUE-15, TIME_TRUE+5, 200)
true_yi = complicated_model_fn(xi, TIME_TRUE, LAMBDA_PLUS_TRUE, LAMBDA_MINUS_TRUE, MAX_AMP_TRUE,
                               OMEGA_0_TRUE, OMEGA_MAX_TRUE, OMEGA_SIGMA_TRUE)

plt.plot(xi, true_yi)
plt.xlabel("Time (s)")
plt.ylabel("Strain");

Let's make some fake data. We'll simulate "noise" as ten sinusoids of varying frequency, phase, and amplitude added together, and superimpose a merger signal at $t=0$.

In [10]:
#>>>RUN

NUMBER_SINES_TO_ADD = 10

noise_frequencies = 0.5 + 7 * np.random.random(NUMBER_SINES_TO_ADD)
noise_phases = 2 * np.pi * np.random.random(NUMBER_SINES_TO_ADD)
noise_amplitudes = 2 * MAX_AMP_TRUE / NUMBER_SINES_TO_ADD * np.random.random(NUMBER_SINES_TO_ADD)
    # The above line sets noise amplitudes so that the sum of all the noise amplitudes is on average
    # equal to the maximum amplitude of the signal.

sample_spacing = 0.1
xi = np.arange(-128, 128, sample_spacing)#times
yi = np.zeros_like(xi)#data

for freq, phase, amplitude in zip(noise_frequencies, noise_phases, noise_amplitudes):
    yi += amplitude * np.sin(phase + freq * xi)
    
yi += complicated_model_fn(xi, TIME_TRUE, LAMBDA_PLUS_TRUE, LAMBDA_MINUS_TRUE, MAX_AMP_TRUE,
                               OMEGA_0_TRUE, OMEGA_MAX_TRUE, OMEGA_SIGMA_TRUE)

plt.figure(figsize=(16, 5))
plt.plot(xi, yi)
plt.xlabel("Time (s)")
plt.ylabel("Strain")

Our goal is to find the signal in this data.

<a name='problems_6_1'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_1) | [Next Section](#section_6_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 6.1.1</span>

text


In [14]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


<a name='section_6_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P6.2 Time Domain</h2>    

| [Top](#section_6_0) | [Previous Section](#section_6_1) | [Problems](#problems_6_2) | [Next Section](#section_6_3) |


<h3>Overview</h3>

Matched filtering in the time domain is probably conceptually the easiest approach to matched filtering. We will perform a fit of the model function to the data, forcing the model function to assume a time of merger of $t$, then plot the quality of the fit as a function of $t$. We expect a very good fit quality when $t$ is close to the true time $t=0$, and otherwise we expect poor fits.

<a name='problems_6_2'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_2) | [Next Section](#section_6_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 6.2.1</span>

We'll need to cut the data to perform the fit. How much time before and after $t$ would you like to fit over?

In [25]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

t_before = #your code here
t_after = #your code here

### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 6.2.2</span>

Make a function that creates `LMFIT` `Model` and `Parameters` instance, forcing the time of the signal to appear at time $t$. Seed the starting points of parameters randomly. Refer to recitation 3 code if you need help.

Restrict your `time` parameter to be exactly $t$ by setting its `value` equal to $t$ and setting `vary=False`

In [25]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

from lmfit import Model, Parameters
    
params_min_max = {
    'lambda_plus': (0.1, 5),
    'lambda_minus': (0.1, 5),
    'max_amp': (0, 2),
    'omega_0': (0, 5),
    'omega_max': (0, 10),
    'omega_sigma': (0, 5)
}

def model_and_random_parameters(t, delta_t):
    #your code here

This function cuts the data to data around 𝑡 and performs a fit, using your model_and_random_parameters function.

In [21]:
#>>>RUN

def fit_once(t):
    data_indices = np.where((xi > t - t_before) & (xi < t + t_after))
    data_x = xi[data_indices]
    data_y = yi[data_indices]
    model, params = model_and_random_parameters(t)
    result = model.fit(data_y, params, x=data_x)
    return result

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 6.2.3</span>

Make a function that creates `LMFIT` `Model` and `Parameters` instance, forcing the time of the signal to appear at time $t$. Seed the starting points of parameters randomly. Refer to recitation 3 code if you need help.

Restrict your `time` parameter to be exactly $t$ by setting its `value` equal to $t$ and setting `vary=False`

In [25]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#your code here

Instead of `fit_once`, we'll use a new function called `fit`, which runs `fit_once` multiple times and takes the best result.

In [21]:
#>>>RUN

from multiprocessing import Pool

NUM_FITS = 6

def fit(t, pool):
    results = pool.map(fit_once, np.full(NUM_FITS, t))
    min_result = None
    min_chisq = None
    for result in results:
        if min_chisq is None or min_chisq > result.chisqr:
            min_result = result
            min_chisq = result.chisqr
    return min_result

with Pool(6) as pool:
    result = fit(TIME_TRUE, pool)
print(result.fit_report())
result.plot();

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 6.2.4</span>

Call `fit` for values of $t \in [-100, 100]$, where the $t$ values are separated by $\Delta t \sim 1 \text{s}$. Store the results in an array named `results`. This may take a while.

In [25]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

results = []
delta_t = 1
ts = np.arange(-10, 10, delta_t)

#your code here

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 6.2.5</span>

Choose some number which represents the success of the fit and plot it as a function of $t$ given the `results` you just calculated.

In [25]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#your code here

### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*This took a while to run. But in practice, it's nice to have searches like this run quickly, so that if a wave event is detected, an alert can be sent out to telescopes all over the world and they can look at the correct area of the sky with minimal delay. How could you make this process faster, aside from running it on better hardware?*</span>


<a name='section_6_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P6.3 Frequency Domain</h2>   

| [Top](#section_6_0) | [Previous Section](#section_6_2) | [Problems](#problems_6_3) |


<h3>Overview</h3>

Now we're going to do a similar process in the frequency domain, but instead of performing a fit, we're going compare the data to a template merger which we will generate.

Let's start by making the template in the time domain. The parameters will be similar to the true values, but slightly off.

In [32]:
#>>>RUN

yi_temp = complicated_model_fn(xi, 0, 1.5, 3.5, 1.0, 3.2, 5.5, 3.5)
yi_true = complicated_model_fn(xi, TIME_TRUE, LAMBDA_PLUS_TRUE, LAMBDA_MINUS_TRUE, MAX_AMP_TRUE,
                               OMEGA_0_TRUE, OMEGA_MAX_TRUE, OMEGA_SIGMA_TRUE)

template_mask = np.where((xi > -15) & (xi < 5))
data_mask = np.where((xi > TIME_TRUE-15) & (xi < TIME_TRUE+5))
plt.plot(xi[data_mask]-TIME_TRUE, yi_true[data_mask], label="True")
plt.plot(xi[template_mask], yi_temp[template_mask], label="Template")
plt.xlabel("Time (s)")
plt.ylabel("Strain")
plt.legend();

We're going to be working in the frequency domain, so start by fft-ing our template and data.

In [32]:
#>>>RUN

import matplotlib.pyplot as plt

fs = int(1/(xi[1] - xi[0]))

data_fft = np.fft.fft(yi)
template_fft = np.fft.fft(yi_temp)

freq = np.fft.fftfreq(xi.shape[0])*fs

#template_fft *= np.exp(-1.0j * 50 * freq * 2 * np.pi)

plt.figure(figsize=(16, 5))
plt.plot(freq, data_fft.real)
plt.plot(freq, data_fft.imag)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.ylim(-20, 20)

plt.figure(figsize=(16, 5))
plt.plot(freq, template_fft.real)
plt.plot(freq, template_fft.imag)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.ylim(-20, 20)

Now suppose we want to do the same thing we did in the time domain case: shift the template by a time $t$. What happens to the FFT? Consider the integral definition of a fourier transform (as a substitute for the discrete case):

$$\mathcal{F}(\omega) = \int dt f(t)  e^{-2\pi i\omega t}$$

So the FFT for a function shifted by $\Delta t$ is

$$\int dt f(t - \Delta t)  e^{-2\pi i\omega t} = \int dt' f(t')  e^{-2\pi i\omega (t'+\Delta t)} = e^{-2\pi i \omega \Delta t}\mathcal{F}(\omega).$$

It's just a multiplicative factor times the initial template FFT!

So suppose our data is indeed just our template, shifted by some time $\Delta t$. We can recover $\Delta t$ from the data FFT $\mathcal{D}$ and the template FFT $\mathcal{T}$ by

$$\mathcal{D}(\omega)=e^{-2\pi i\omega \Delta t} \mathcal{T}(\omega) \implies \frac{\mathcal{D}(\omega)}{\mathcal{T}(\omega)}=e^{-2\pi i\omega \Delta t} = \frac{\mathcal{D}(\omega) \mathcal{T}^*(\omega)}{|\mathcal{T}|^2(\omega)}.$$

Take the inverse Fourier transform of this ratio and you get a delta function at $\Delta t$:

$$f(t) = \int d\omega e^{-2\pi i\omega \Delta t} e^{2\pi i\omega t} = \delta(\Delta t - t).$$

In other words, the peak of the IFFT of 
$$f(\omega) =\mathcal{D}(\omega) \mathcal{T}^*(\omega)$$
is centered on $\Delta t$.

There's some extra complication behind this, but above is the gist. For more information, see the [LIGO tutorial from which this code is inspired](https://www.gw-openscience.org/s/events/GW150914/LOSC_Event_tutorial_GW150914.html).

Let's compute $f(\omega)$ as written above, using the power spectral density of the data instead of $|\mathcal{T}|^2$ in the numerator. We call it `optimal_fft`. Then we take the IFFT and call it `optimal_time`.

In [32]:
#>>>RUN

power_data, freq_psd = plt.psd(yi, Fs=fs, NFFT=4 * fs)
power_vec = np.interp(freq, freq_psd, power_data)

optimal_fft = data_fft * template_fft.conjugate() / power_vec

optimal_time = 2*np.fft.ifft(optimal_fft)*fs
optimal_time_xi = np.append(xi[len(xi)//2:], xi[:len(xi)//2])

We should really plot some sort of signal to noise ratio, not `optimal_time` itself. Let's fix the ratio so that if the data is just noise, we have a ratio of one. We record the ratio as `SNR` and plot it.

In [32]:
#>>>RUN

df = np.abs(freq[1] - freq[0])
sigmasq = 2*(template_fft * template_fft.conjugate() / power_vec).sum() * df
sigma = np.sqrt(np.abs(sigmasq))
SNR = abs(optimal_time) / (sigma)

plt.figure()
plt.plot(optimal_time_xi, SNR)
plt.xlabel('Offset time (s)')
plt.ylabel('SNR');

### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*Change the data, perhaps by adding more noise or a different kind of noise, or by shifting the signal to a different time. What changes in the matched filter result?*</span>


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*The runtime of the frequency domain matched filtering process was much faster than the time domain process. But we should not compare the directly because we tested against a template for the frequency version and performed a whole fit for the time version.*</span>
>
><span style="color: #90409C;">*If we instead used a template for the time version as well, computing a $\chi^2$ for every offset $t$ between the signal and the template and plotted $\chi^2$ as a function of $t$, which method do you think would be faster now? Frequency or time?*</span>


<a name='problems_6_3'></a>   

| [Top](#section_6_0) | [Restart Section](#section_6_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 6.3.1</span>

text


In [36]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


### <span style="color: #90409C;">*>>> Follow-up (ungraded)*</span>

    
><span style="color: #90409C;">*TEXT*</span>
